In [114]:
import pandas as pd
from feature_engine.encoding import WoEEncoder, RareLabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import RFE,SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from xverse.transformer import WOE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import numpy as np

In [115]:
df=pd.read_csv('train.csv')

In [116]:
# Null values.
df.isnull().sum()/len(df)*100

Year                         0.000000
Month                        0.000000
ProductCategory              0.000000
Sales(In ThousandDollars)    5.555556
dtype: float64

In [117]:
df.fillna(df.mean(),inplace=True)

C:\Users\mahes\AppData\Local\Temp\ipykernel_9328\2085774198.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.fillna(df.mean(),inplace=True)


In [118]:
df.isnull().sum()

Year                         0
Month                        0
ProductCategory              0
Sales(In ThousandDollars)    0
dtype: int64

In [119]:
X_train,y_train=df.drop(columns=['Sales(In ThousandDollars)']),df['Sales(In ThousandDollars)']

In [120]:
X_test=pd.read_csv('submission.csv')
X_test.head(3)

,Year,Month,ProductCategory,Unnamed: 3,Sales(In ThousandDollars)
0,2014,1,WomenClothing,NaN,NaN
1,2014,1,MenClothing,NaN,NaN
2,2014,1,OtherClothing,NaN,NaN


In [121]:
X_test.drop(columns=['Unnamed: 3','Sales(In ThousandDollars)'],inplace=True)

In [122]:
# Cat columns
X_cat=df.select_dtypes(include='object').columns

In [123]:
X_train.head(5)

,Year,Month,ProductCategory
0,2009,1,WomenClothing
1,2009,1,MenClothing
2,2009,1,OtherClothing
3,2009,2,WomenClothing
4,2009,2,MenClothing


In [124]:
oe=LabelEncoder()
X_train['Year']=oe.fit_transform(X_train['Year'])
X_train['ProductCategory']=oe.fit_transform(X_train['ProductCategory'])
X_train.head()

,Year,Month,ProductCategory
0,0,1,2
1,0,1,0
2,0,1,1
3,0,2,2
4,0,2,0


In [125]:
X_test['Year']=X_test['Year'].map(lambda x: 5 if x==2014 else 0)
X_test['ProductCategory']=oe.fit_transform(X_test['ProductCategory'])
X_test.head()

,Year,Month,ProductCategory
0,5,1,2
1,5,1,0
2,5,1,1
3,5,2,2
4,5,2,0


In [126]:
# Model Building
linereg=LinearRegression()
linereg.fit(X_train,y_train)
y_train_pred=linereg.predict(X_train)
print("Training R2")
print(linereg.score(X_train,y_train))
rmse_train=mean_squared_error(y_train,y_train_pred,squared=False)
print('RMSE: ',rmse_train,3)

Training R2
0.762267492902619
RMSE:  547.1974901410885 3


In [127]:
# predict
y_pred_train=linereg.predict(X_train)
y_pred_test=linereg.predict(X_test)

In [128]:
y_pred_test

array([2900.9564178 ,  575.22308446, 1738.08975113, 2948.04614219,
        622.31280886, 1785.17947552, 2995.13586658,  669.40253325,
       1832.26919992, 3042.22559098,  716.49225764, 1879.35892431,
       3089.31531537,  763.58198204, 1926.4486487 , 3136.40503976,
        810.67170643, 1973.5383731 , 3183.49476416,  857.76143082,
       2020.62809749, 3230.58448855,  904.85115522, 2067.71782188,
       3277.67421294,  951.94087961, 2114.80754628, 3324.76393734,
        999.030604  , 2161.89727067, 3371.85366173, 1046.1203284 ,
       2208.98699506, 3418.94338612, 1093.21005279, 2256.07671946])

In [129]:
from sklearn.linear_model import SGDRegressor

In [130]:
sgd_model=SGDRegressor()
sgd_model.fit(X_train,y_train)

SGDRegressor()

In [131]:
y_pred=sgd_model.predict(X_test)
y_pred

array([2898.317573  ,  632.63049222, 1765.47403261, 2931.79000475,
        666.10292397, 1798.94646436, 2965.2624365 ,  699.57535572,
       1832.41889611, 2998.73486825,  733.04778747, 1865.89132786,
       3032.2073    ,  766.52021922, 1899.36375961, 3065.67973175,
        799.99265097, 1932.83619136, 3099.1521635 ,  833.46508271,
       1966.30862311, 3132.62459525,  866.93751446, 1999.78105485,
       3166.09702699,  900.40994621, 2033.2534866 , 3199.56945874,
        933.88237796, 2066.72591835, 3233.04189049,  967.35480971,
       2100.1983501 , 3266.51432224, 1000.82724146, 2133.67078185])

In [132]:
y_pred.shape

(36,)

In [133]:
X_test['Sales(In ThousandDollars)']=y_pred

In [134]:
X_test.shape

(36, 4)

In [135]:
#X_test.reset_index()

In [136]:
X_test2 = X_test.drop(columns=['Year','Month','ProductCategory']).reset_index().rename({'index':'Year'},axis=1)

In [137]:
X_test2.shape

(36, 2)

In [138]:
X_test2.to_csv('Submission_MR.csv',index=False)